#### Test the downloaded file

In [18]:
import pandas as pd
import io
import requests
url="https://archive.ics.uci.edu/ml/machine-learning-databases/00639/Maternal%20Health%20Risk%20Data%20Set.csv"
#s=requests.get(url).content
#c=pd.read_csv(io.StringIO(s.decode('utf-8')))
x=pd.read_csv(url)
y = x.pop("RiskLevel")    

In [20]:
x[:2]

,Age,SystolicBP,DiastolicBP,BS,BodyTemp,HeartRate
0,25,130,80,15.0,98.0,86
1,35,140,90,13.0,98.0,70


In [21]:
y[:2]

0    high risk
1    high risk
Name: RiskLevel, dtype: object

In [22]:
y.unique()

array(['high risk', 'low risk', 'mid risk'], dtype=object)

# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
from azureml.core import Workspace, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os
import pandas as pd
from sklearn.model_selection import train_test_split
#from azureml.core import Dataset
from azureml.train.automl import AutoMLConfig
from azureml.automl.runtime.onnx_convert import OnnxConverter

2022-03-16:21:31:39,878 INFO     [modeling_bert.py:226] Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex .
2022-03-16:21:31:39,883 INFO     [modeling_xlnet.py:339] Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex .


# Create or Attach an AmlCompute cluster

In [2]:
ws = Workspace.from_config()
experiment_name = 'capstone-hyperdrive-experiment'

experiment=Experiment(ws, experiment_name)


print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

2022-03-16:21:31:40,822 INFO     [workspace.py:290] Found the config file in: /config.json
2022-03-16:21:31:41,230 INFO     [_universal.py:475] Request URL: 'https://management.azure.com/subscriptions?api-version=REDACTED'/nRequest method: 'GET'/nRequest headers:/n    'Accept': 'application/json'/n    'x-ms-client-request-id': '77f427fe-a570-11ec-8beb-0bf01a288124'/n    'User-Agent': 'azsdk-python-azure-mgmt-resource/20.0.0 Python/3.6.9 (Linux-5.4.0-1068-azure-x86_64-with-debian-buster-sid)'/n    'Authorization': 'REDACTED'/nNo body was attached to the request
2022-03-16:21:31:41,574 INFO     [_universal.py:503] Response status: 200/nResponse headers:/n    'Cache-Control': 'no-cache'/n    'Pragma': 'no-cache'/n    'Content-Type': 'application/json; charset=utf-8'/n    'Content-Encoding': 'REDACTED'/n    'Expires': '-1'/n    'Vary': 'REDACTED'/n    'x-ms-ratelimit-remaining-tenant-reads': '11999'/n    'x-ms-request-id': 'fbec4a9d-c73e-43a0-9776-b8d4eb5218b8'/n    'x-ms-correlation-reque

quick-starts-ws-189146
aml-quickstarts-189146
southcentralus
5a4ab2ba-6c51-4805-8155-58759ad589d8


In [3]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

# NOTE: update the cluster name to match the existing cluster
# Choose a name for your CPU cluster
amlcompute_cluster_name = "automl-com-clst" #automl-com-clst #automl-com-clst

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',# for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 10)
# For a more detailed view of current AmlCompute status, use get_status().

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [4]:
# Try to load the dataset from the Workspace. Otherwise, create it from the file
# NOTE: update the key to match the dataset name
found = False
#key = "BankMarketing Dataset"
key="Maternal_Health_Risk_Data_Set" #"bankmarketing_train" #bankmarketing_train
#bankmarketing_train
description_text = "UCI machine Learning Maternal Health Risk Data Set"

if key in ws.datasets.keys(): 
        found = True
        dataset = ws.datasets[key] 
#https://archive.ics.uci.edu/ml/machine-learning-databases/00639/Maternal%20Health%20Risk%20Data%20Set.csv
#https://github.com/ShubraChowdhury/AzureMLCapstone/blob/master/starter_file/Maternal_Health_Risk_Data_Set.csv

if not found:
        # Create AML Dataset and register it into Workspace
        example_data = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00639/Maternal%20Health%20Risk%20Data%20Set.csv'
        dataset = Dataset.Tabular.from_delimited_files(example_data)        
        #Register Dataset in Workspace
        dataset = dataset.register(workspace=ws,
                                   name=key,
                                   description=description_text)


df = dataset.to_pandas_dataframe()
df.describe()

,Age,SystolicBP,DiastolicBP,BS,BodyTemp,HeartRate
count,1014.00,1014.00,1014.00,1014.00,1011.00,1014.00
mean,29.87,113.20,76.46,8.73,98.67,74.30
std,13.47,18.40,13.89,3.29,1.37,8.09
min,10.00,70.00,49.00,6.00,98.00,7.00
25%,19.00,100.00,65.00,6.90,98.00,70.00
50%,26.00,120.00,80.00,7.50,98.00,76.00
75%,39.00,120.00,90.00,8.00,98.00,80.00
max,70.00,160.00,100.00,19.00,103.00,90.00


Review the Dataset Result

In [5]:
dataset.take(5).to_pandas_dataframe()

,Age,SystolicBP,DiastolicBP,BS,BodyTemp,HeartRate,RiskLevel
0,25,130,80,15.00,98,86,high risk
1,35,140,90,13.00,98,70,high risk
2,29,90,70,8.00,100,80,high risk
3,30,140,85,7.00,98,70,high risk
4,35,120,60,6.10,98,76,low risk


In [7]:
from sklearn.model_selection import train_test_split
from azureml.data.dataset_factory import TabularDatasetFactory
#TRAIN_d, TEST_d = train_test_split(df, shuffle=True)
TRAIN_d, TEST_d = train_test_split(df, test_size=0.2, random_state=24, shuffle=True)
TRAIN_d.to_csv('train.csv',index = False)

ds_new = ws.get_default_datastore()
ds_new.upload_files(files = ['./train.csv'])
#ds_new.upload(src_dir='.',target_path='.')


train_data =  TabularDatasetFactory.from_delimited_files(path=[(ds_new,('train.csv'))])

2022-03-16:21:35:24,501 INFO     [datastore_client.py:908] <azureml.core.authentication.InteractiveLoginAuthentication object at 0x7f998556d080>
2022-03-16:21:35:24,622 INFO     [azure_storage_datastore.py:832] Called AzureBlobDatastore.upload_files
2022-03-16:21:35:24,623 INFO     [azure_storage_datastore.py:370] Uploading an estimated of 1 files
2022-03-16:21:35:24,701 INFO     [azure_storage_datastore.py:370] Target already exists. Skipping upload for train.csv
2022-03-16:21:35:24,702 INFO     [azure_storage_datastore.py:370] Uploaded 0 files
2022-03-16:21:35:24,703 INFO     [azure_storage_datastore.py:842] Finished AzureBlobDatastore.upload with count=0.


## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [14]:
from sklearn.model_selection import train_test_split
# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
#BanditPolicy(evaluation_interval=1, slack_factor=None, slack_amount=None, delay_evaluation=0)
early_termination_policy = BanditPolicy(evaluation_interval=1, slack_factor=0.1)

#TODO: Create the different params that you will be using during training
#solver{‘newton-cg’, ‘lbfgs’, ‘liblinear’, ‘sag’, ‘saga’}, default=’lbfgs’
ps = RandomParameterSampling(
    {
"--C" : choice(0.01,0.1,1) ,     
"--max_iter" : choice(10,20,40),
"--solver" :  choice('newton-cg','lbfgs','liblinear','sag', 'saga')        
}
)  


#TODO: Create your estimator and hyperdrive config


if "training" not in os.listdir():
    os.mkdir("./training")

# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

# Create a ScriptRunConfig Object to specify the configuration details of your training job
#SKLearn(source_directory, *, compute_target=None, vm_size=None, vm_priority=None, entry_script=None, script_params=None, use_docker=True, custom_docker_image=None, image_registry_details=None, user_managed=False, conda_packages=None, pip_packages=None, conda_dependencies_file_path=None, pip_requirements_file_path=None, conda_dependencies_file=None, pip_requirements_file=None, environment_variables=None, environment_definition=None, inputs=None, shm_size=None, resume_from=None, max_run_duration_seconds=None, framework_version=None, _enable_optimized_mode=False, _disable_validation=True, _show_lint_warnings=False, _show_package_warnings=False)
estimator = SKLearn(source_directory="./",entry_script='train.py',compute_target=amlcompute_cluster_name)



hyperdrive_run_config =  HyperDriveConfig(
        hyperparameter_sampling=ps,
        primary_metric_name='Accuracy',
        primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
        max_total_runs = 10, 
        max_concurrent_runs =4,
        policy=early_termination_policy,
        estimator=estimator

)

2022-03-16:21:41:40,63 WARNING  [sklearn.py:256] 'SKLearn' estimator is deprecated. Please use 'ScriptRunConfig' from 'azureml.core.script_run_config' with your own defined environment or the AzureML-Tutorial curated environment.


In [16]:
#TODO: Submit your experiment

hyperdrive_run=experiment.submit(hyperdrive_run_config)


2022-03-16:21:42:54,54 WARNING  [script_run_config.py:375] If 'script' has been provided here and a script file name has been specified in 'run_config', 'script' provided in ScriptRunConfig initialization will take precedence.
2022-03-16:21:42:55,645 WARNING  [connectionpool.py:304] Connection pool is full, discarding connection: mlstrg189146.blob.core.windows.net
2022-03-16:21:42:55,649 WARNING  [connectionpool.py:304] Connection pool is full, discarding connection: mlstrg189146.blob.core.windows.net


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [17]:
RunDetails(hyperdrive_run).show()
hyperdrive_run.get_status()
hyperdrive_run.wait_for_completion(show_output=True)

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_0aa44941-4268-41ad-a8ef-678b537706e5
Web View: https://ml.azure.com/runs/HD_0aa44941-4268-41ad-a8ef-678b537706e5?wsid=/subscriptions/5a4ab2ba-6c51-4805-8155-58759ad589d8/resourcegroups/aml-quickstarts-189146/workspaces/quick-starts-ws-189146&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

"<START>[2022-03-16T21:42:56.583383][API][INFO]Experiment created<END>\n""<START>[2022-03-16T21:42:57.782777][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space<END>\n""<START>[2022-03-16T21:42:58.111890][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.<END>\n"


## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [ ]:
### YOUR CODE HERE ###
best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics() 

print("Best Run ID :", best_run.id)
print("\n Accuracy :", best_run_metrics['Accuracy'])

print("\n best run file names :",best_run.get_file_names())
print("\n best run details ", best_run.get_details())
print("\n best run metrics :",best_run_metrics)

In [ ]:
#TODO: Save the best model
import joblib

model = best_run.register_model(model_name='best_hyperdrive_model', model_path='outputs/best_hyperdrive_model.joblib')

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [ ]:
from azureml.core.model import InferenceConfig
from azureml.core.webservice import Webservice, AciWebservice

inference_config = InferenceConfig(entry_script="score.py", environment = sklearn_env)

deployment_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb = 1,enable_app_insights = True)

In [ ]:
service=Model.deploy(workspace=ws,
                    name="Meternal-health-hyper-deploy-service",
                    models=[model],
                    inference_config=inference_config,
                    deployment_config=deployment_config)
service.wait_for_deployment(show_output=True)

TODO: In the cell below, send a request to the web service you deployed to test it.

In [ ]:
scoring_uri = service.scoring_uri

print(f'\nservice state: {service.state}\n')
print(f'scoring URI: \n{service.scoring_uri}\n')
print(f'swagger URI: \n{service.swagger_uri}\n')

print(service.scoring_uri)
print(service.swagger_uri)

In [ ]:
POST_TEST_DATA= TEST_d[:2][:2].values.tolist()
POST_TEST_DATA = [POST_TEST_DATA[0][:-1],POST_TEST_DATA[1][:-1]]

TODO: In the cell below, print the logs of the web service and delete the service

In [ ]:
import requests
import json

data = json.dumps({"data":POST_TEST_DATA})
headers = {'Content-Type':'application/json'}

response = requests.post(scoring_uri,data,headers=headers)
print(response.text)

In [ ]:
print('Status code:',response.status_code)

print(service.get_logs())

In [ ]:
model.delete()
service.delete()
compute_target.delete()


**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.

